In [1]:
from pyspark.sql import SparkSession

In [2]:
working_directory = '/home/spark/Downloads/rapids-4-spark_2.12-23.02.0.jar,/home/spark/spark/jars/mongo-spark-connector/bson-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/bson-record-codec-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/mongodb-driver-core-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/mongodb-driver-sync-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/mongo-spark-connector_2.12-10.1.1.jar'


spark = SparkSession \
    .builder \
    .appName("myApp") \
    .master("local[3]") \
    .config("spark.driver.memory", "2G") \
    .config("spark.executor.memory", "2G") \
    .config("spark.mongodb.read.connection.uri=mongodb://10.122.6.17:27017/vnw_job.events") \
    .config("spark.mongodb.write.connection.uri=mongodb://10.122.6.17:27017/vnw_job.events") \
    .config("spark.jars", "/home/spark/Downloads/rapids-4-spark_2.12-23.02.0.jar,/home/spark/spark/jars/mongo-spark-connector/bson-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/bson-record-codec-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/mongodb-driver-core-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/mongodb-driver-sync-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/mongo-spark-connector_2.12-10.1.1.jar,/home/spark/Downloads/spark-tensorflow-connector_2.11-1.14.0.jar")\
    .getOrCreate()

# spark.stop()

24/03/20 10:00:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# spark.stop()

In [43]:
'''MySQL'''
import pandas as pd
import mysql.connector
# database = "vnw_core"
database = 'in_inter'
host = "172.16.1.75"

cnx = mysql.connector.connect(
    host=host,
    port="3306",
    user='ylv', 
    password="b693DTDoud", 
    database=database
)

query = """select * from in_inter.tblresume_industry_sample_20230705 where checker=1;"""

cursor = cnx.cursor()
cursor.execute(query)
data = cursor.fetchall()
df_sql = pd.DataFrame(data, columns=[i[0] for i in cursor.description])
cursor.close()

True

In [44]:
df_sql

,resumeid,userid,resumetitle,desiredjobtitle,mostrecentposition,mostrecentemployer,industryOld,industryV3Name,checker
0,266457,815561,2022-03-15 05:59:46 Sang Tôn Thất,MEP Project Manager,MEP Project Manager,Foreigner Company,Executive management,Real Estate/Rental/Leasing,1
1,484328,417254,2019/04/04 00:30:23 Phi Hùng Huỳnh,Chuyên viên kinh doanh - Dịch vụ,Customer Service/ Sales/ Marketing,"From 2005 to now, working for some companies.","Customer Service,Sales,Sales Technical",Automotive,1
2,538689,1155840,2023-07-06 06:59:47 Thị Duy Linh Bùi,Phiên dịch tự do,Phiên dịch tự do,Phiên dịch tự do,"Administrative/Clerical,Sales,Interpreter/Tran...",Media/Newspaper/Advertising,1
3,1711433,2412350,2023-04-11 22:41:45 KIEN NGUYEN,Procurement Manager,Procurement Manager,Nova Service Group,Merchandising/Purchasing/Supply Chain,FMCG,1
4,1835176,1257373,2023-05-24 20:21:33 Đạt Võ,Senior Sales Executive,Senior Sales Executive,"Fujitec Vietnam Co., Ltd.","Sales,Sales Technical",Electrical/Electronics,1
...,...,...,...,...,...,...,...,...,...
354,10497849,7286334,2023-07-06 08:35:29 Quế Anh Lưu Dương,Sinh Viên,Tình nguyện viên,LIFT charitable organization,Entry level,IT Software/SaaS,1
355,10497896,7285717,CV_Lê Thị Thảo Sương.pdf-2023-07-06,Phó phòng kế hoạch,Phó phòng Kế hoạch,Công ty Trách nhiệm hữu hạn một thành viên Ô t...,Other,Manufacturing,1
356,10497903,7285793,CURRICULUM VITAE.docx-2023-07-06,Trưởng phòng kinh doanh,None,None,Sales,Manufacturing,1
357,10497921,7286336,kỹ sư an toàn,kỹ sư an toàn,None,công ty NOVALAND,HSE,Manufacturing,1


In [16]:
users = list(map(str,df_sql.userid.unique().tolist()))

In [32]:
from pymongo import MongoClient

con = MongoClient("mongodb://10.122.6.17:27017")
cur = con.vnw_job.events.find({'entityId': {'$in': users}}, {"entityId": 1, "targetEntityId": 1, "event": 1})

In [33]:
df = pd.DataFrame(list(cur))

In [35]:
df.to_csv('./mapping_industries/resume_data.csv', index=False)

In [4]:
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType

# schema = StructType([
#     StructField("entityId", StringType(), True),
#     StructField("targetEntityId", StringType(), True)
# ])

# df = spark.read.format("csv").load("/home/spark/ylv/data/navidata/*", schema=schema)

In [5]:
# df.show(10)

+--------------------+--------------+
|            entityId|targetEntityId|
+--------------------+--------------+
|             6612804|       1624701|
|52.9.73.55, 10.12...|       1619921|
|             5132497|       1626036|
|             7086582|       1629851|
|             5291876|       1620867|
|             1842295|       1621413|
|68.169.42.157, 10...|       1630422|
|             7075271|       1630145|
|fdb8:b0b8:7f0:0:c...|       1630342|
|             4764495|       1627604|
+--------------------+--------------+
only showing top 10 rows



In [6]:
df.count()

10040693

In [3]:
# df = spark.read.format("mongodb").option("database", "vnw_job").option("collection", "events").option("pipeline", '[{"$match": {"eventTime": {"$gte": ISODate("2023-04-01T17:00:00Z"), "$lte": ISODate("2023-04-02T17:00:00Z")}}}]').load()
# df = spark.read.format("mongodb").option("database", "data_6").option("collection", "events").load()

from pyspark.sql.types import StructType,StructField, StringType, IntegerType

schema = StructType([
    StructField("entityId", StringType(), True),
    StructField("targetEntityId", StringType(), True),
    StructField("event", StringType(), True)
])

%time df = spark.read.format("mongodb").option("spark.mongodb.database", "vnw_job").option("spark.mongodb.collection", "events").option("aggregation.pipeline", '[{"$match": {"eventTime": {"$gte": ISODate("2024-03-01T00:00:00Z"), "$lt": ISODate("2024-04-01T00:00:00Z")}}}, {"$project": {"entityId": 1, "targetEntityId": 1, "event": 1}}]').load(schema=schema)

# %time df = spark.read.format("mongodb").option("spark.mongodb.database", "vnw_job").option("spark.mongodb.collection", "events").option("aggregation.pipeline", '[{"$match": {"eventTime": {"$gte": ISODate("2023-06-01T00:00:00Z")}}}, {"$project": {"entityId": 1, "targetEntityId": 1, "event": 1}}]').load(schema=schema)
# %time df = spark.read.format("mongodb").option("spark.mongodb.database", "vnw_job").option("spark.mongodb.collection", "events").option("aggregation.pipeline", '[{"$match": {"eventTime": {"$gte": ISODate("2023-07-01T00:00:00Z"))}}}, {"$project": {"entityId": 1, "targetEntityId": 1, "event": 1}}]').load(schema=schema)
# %time df = spark.read.format("mongodb").option("spark.mongodb.database", "vnw_job").option("spark.mongodb.collection", "events").option("aggregation.pipeline", '[{"$match": {"entityId": {"$in": ["815561", "417254"], "$lt": ISODate("2023-01-01T00:00:00Z")}}}, {"$project": {"entityId": 1, "targetEntityId": 1, "event": 1}}]').load(schema=schema)

CPU times: user 1.23 ms, sys: 1.15 ms, total: 2.38 ms
Wall time: 1.54 s


In [5]:
# df.count()

8395562

In [4]:
df.write.format('parquet').save('/home/spark/ylv/workplace/mongo_data/032024.parquet', schema=schema)

In [7]:
import os
import tempfile

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

plt.style.use('seaborn-whitegrid')

2023-04-11 15:31:51.951589: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/spark/miniconda3/envs/recommend/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1699614/4133561072.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [8]:
%%time

unique_userids = df.select("entityId").distinct().collect()
unique_itemids = df.select("targetEntityId").distinct().collect()

CPU times: user 1.41 s, sys: 51.5 ms, total: 1.46 s
Wall time: 5.54 s


In [9]:
movies = tf.data.Dataset.from_tensor_slices({
    "job_id": unique_itemids
})

2023-04-11 15:32:40.685238: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-11 15:32:40.685264: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: 172.18.3.41
2023-04-11 15:32:40.685269: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: 172.18.3.41
2023-04-11 15:32:40.685325: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.39.1
2023-04-11 15:32:40.685341: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.39.1
2023-04-11 15:32:40.685345: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.39.1
2023-04-11 15:32:40.685625: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with o

In [10]:
# tensor_slices = {
#     "user_id": df.select("entityId").collect(),
#     "job_id": df.select("targetEntityId")
# }
userids = df.select("entityId").collect()

In [20]:
data = tf.data.Dataset.from_tensor_slices({"userid": userids[:10]})

In [21]:
for i in data.take(1):
    print(i)

{'userid': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'6612804'], dtype=object)>}


In [9]:
itemids = df.select("targetEntityId").collect()

In [ ]:
# userids = df.select("targetEnityId").collect()
data = tf.data.Dataset.from_tensor_slices(tensor_slices)

2023-04-11 14:36:06.822522: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-11 14:36:06.847491: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-11 14:36:06.847682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-11 14:36:06.848143: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [ ]:
ratings = tf.data.Dataset.from_tensor_slices(tensor_slices)

In [14]:
df.write.format("csv").save("/home/spark/ylv/data/navidata_csv_1/navidata.csv", schema=schema)

23/04/10 15:56:54 INFO V2ScanRelationPushDown: 
Output: entityId#0, targetEntityId#1
         
23/04/10 15:56:54 INFO Partitioner: Getting collection stats for: vnw_job.events
23/04/10 15:56:54 INFO client: MongoClient with metadata {"driver": {"name": "mongo-java-driver|sync|mongo-spark-connector|source", "version": "4.8.2|10.1.1"}, "os": {"type": "Linux", "name": "Linux", "architecture": "amd64", "version": "5.15.0-60-generic"}, "platform": "Java/Ubuntu/11.0.17+8-post-Ubuntu-1ubuntu220.04|Scala/2.12.15/Spark/3.2.3"} created with settings MongoClientSettings{readPreference=primary, writeConcern=WriteConcern{w=null, wTimeout=null ms, journal=null}, retryWrites=true, retryReads=true, readConcern=ReadConcern{level=null}, credential=null, streamFactoryFactory=null, commandListeners=[], codecRegistry=ProvidersCodecRegistry{codecProviders=[ValueCodecProvider{}, BsonValueCodecProvider{}, DBRefCodecProvider{}, DBObjectCodecProvider{}, DocumentCodecProvider{}, CollectionCodecProvider{}, Iterab

23/04/10 15:58:33 INFO BlockManagerInfo: Removed broadcast_9_piece0 on localhost:36465 in memory (size: 80.2 KiB, free: 434.3 MiB)
23/04/10 15:58:42 INFO FileOutputCommitter: Saved output of task 'attempt_202304101558056548402006485400041_0012_m_000011_315' to file:/home/spark/ylv/data/navidata_csv_1/navidata.csv/_temporary/0/task_202304101558056548402006485400041_0012_m_000011
23/04/10 15:58:42 INFO SparkHadoopMapRedUtil: attempt_202304101558056548402006485400041_0012_m_000011_315: Committed
23/04/10 15:58:42 INFO Executor: Finished task 11.0 in stage 12.0 (TID 315). 2515 bytes result sent to driver
23/04/10 15:58:42 INFO TaskSetManager: Starting task 12.0 in stage 12.0 (TID 316) (localhost, executor driver, partition 12, ANY, 5023 bytes) taskResourceAssignments Map()
23/04/10 15:58:42 INFO Executor: Running task 12.0 in stage 12.0 (TID 316)
23/04/10 15:58:42 INFO TaskSetManager: Finished task 11.0 in stage 12.0 (TID 315) in 37702 ms on localhost (executor driver) (1/36)
23/04/10 15:5

23/04/10 15:58:58 INFO FileOutputCommitter: Saved output of task 'attempt_202304101558055528174118103338732_0012_m_000004_308' to file:/home/spark/ylv/data/navidata_csv_1/navidata.csv/_temporary/0/task_202304101558055528174118103338732_0012_m_000004
23/04/10 15:58:58 INFO SparkHadoopMapRedUtil: attempt_202304101558055528174118103338732_0012_m_000004_308: Committed
23/04/10 15:58:58 INFO Executor: Finished task 4.0 in stage 12.0 (TID 308). 2515 bytes result sent to driver
23/04/10 15:58:58 INFO TaskSetManager: Starting task 14.0 in stage 12.0 (TID 318) (localhost, executor driver, partition 14, ANY, 5023 bytes) taskResourceAssignments Map()
23/04/10 15:58:58 INFO Executor: Running task 14.0 in stage 12.0 (TID 318)
23/04/10 15:58:58 INFO TaskSetManager: Finished task 4.0 in stage 12.0 (TID 308) in 53647 ms on localhost (executor driver) (3/36)
23/04/10 15:58:58 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
23/04/10 15:58:58 INFO FileOutputCommitter: FileOutputCom

23/04/10 15:59:13 INFO FileOutputCommitter: Saved output of task 'attempt_202304101558056421208899476067275_0012_m_000010_314' to file:/home/spark/ylv/data/navidata_csv_1/navidata.csv/_temporary/0/task_202304101558056421208899476067275_0012_m_000010
23/04/10 15:59:13 INFO SparkHadoopMapRedUtil: attempt_202304101558056421208899476067275_0012_m_000010_314: Committed
23/04/10 15:59:13 INFO Executor: Finished task 10.0 in stage 12.0 (TID 314). 2515 bytes result sent to driver
23/04/10 15:59:13 INFO TaskSetManager: Starting task 16.0 in stage 12.0 (TID 320) (localhost, executor driver, partition 16, ANY, 5023 bytes) taskResourceAssignments Map()
23/04/10 15:59:13 INFO Executor: Running task 16.0 in stage 12.0 (TID 320)
23/04/10 15:59:13 INFO TaskSetManager: Finished task 10.0 in stage 12.0 (TID 314) in 68267 ms on localhost (executor driver) (5/36)
23/04/10 15:59:13 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
23/04/10 15:59:13 INFO FileOutputCommitter: FileOutputC

23/04/10 15:59:26 INFO FileOutputCommitter: Saved output of task 'attempt_202304101558055846412203832210945_0012_m_000005_309' to file:/home/spark/ylv/data/navidata_csv_1/navidata.csv/_temporary/0/task_202304101558055846412203832210945_0012_m_000005
23/04/10 15:59:26 INFO SparkHadoopMapRedUtil: attempt_202304101558055846412203832210945_0012_m_000005_309: Committed
23/04/10 15:59:26 INFO Executor: Finished task 5.0 in stage 12.0 (TID 309). 2515 bytes result sent to driver
23/04/10 15:59:26 INFO TaskSetManager: Starting task 18.0 in stage 12.0 (TID 322) (localhost, executor driver, partition 18, ANY, 5023 bytes) taskResourceAssignments Map()
23/04/10 15:59:26 INFO Executor: Running task 18.0 in stage 12.0 (TID 322)
23/04/10 15:59:26 INFO TaskSetManager: Finished task 5.0 in stage 12.0 (TID 309) in 80940 ms on localhost (executor driver) (7/36)
23/04/10 15:59:26 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
23/04/10 15:59:26 INFO FileOutputCommitter: FileOutputCom

...

23/04/10 16:00:17 INFO FileOutputCommitter: Saved output of task 'attempt_202304101558055248447893918576980_0012_m_000013_317' to file:/home/spark/ylv/data/navidata_csv_1/navidata.csv/_temporary/0/task_202304101558055248447893918576980_0012_m_000013
23/04/10 16:00:17 INFO SparkHadoopMapRedUtil: attempt_202304101558055248447893918576980_0012_m_000013_317: Committed
23/04/10 16:00:17 INFO Executor: Finished task 13.0 in stage 12.0 (TID 317). 2515 bytes result sent to driver
23/04/10 16:00:17 INFO TaskSetManager: Starting task 26.0 in stage 12.0 (TID 330) (localhost, executor driver, partition 26, ANY, 5023 bytes) taskResourceAssignments Map()
23/04/10 16:00:17 INFO Executor: Running task 26.0 in stage 12.0 (TID 330)
23/04/10 16:00:17 INFO TaskSetManager: Finished task 13.0 in stage 12.0 (TID 317) in 93841 ms on localhost (executor driver) (15/36)
23/04/10 16:00:17 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
23/04/10 16:00:17 INFO FileOutputCommitter: FileOutput

23/04/10 16:00:45 INFO FileOutputCommitter: Saved output of task 'attempt_20230410155805398690714149351162_0012_m_000015_319' to file:/home/spark/ylv/data/navidata_csv_1/navidata.csv/_temporary/0/task_20230410155805398690714149351162_0012_m_000015
23/04/10 16:00:45 INFO SparkHadoopMapRedUtil: attempt_20230410155805398690714149351162_0012_m_000015_319: Committed
23/04/10 16:00:45 INFO Executor: Finished task 15.0 in stage 12.0 (TID 319). 2515 bytes result sent to driver
23/04/10 16:00:45 INFO TaskSetManager: Starting task 28.0 in stage 12.0 (TID 332) (localhost, executor driver, partition 28, ANY, 5023 bytes) taskResourceAssignments Map()
23/04/10 16:00:45 INFO Executor: Running task 28.0 in stage 12.0 (TID 332)
23/04/10 16:00:45 INFO TaskSetManager: Finished task 15.0 in stage 12.0 (TID 319) in 105722 ms on localhost (executor driver) (17/36)
23/04/10 16:00:45 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
23/04/10 16:00:45 INFO FileOutputCommitter: FileOutputCo

23/04/10 16:00:57 INFO FileOutputCommitter: Saved output of task 'attempt_202304101558059220934181039852711_0012_m_000024_328' to file:/home/spark/ylv/data/navidata_csv_1/navidata.csv/_temporary/0/task_202304101558059220934181039852711_0012_m_000024
23/04/10 16:00:57 INFO SparkHadoopMapRedUtil: attempt_202304101558059220934181039852711_0012_m_000024_328: Committed
23/04/10 16:00:57 INFO Executor: Finished task 24.0 in stage 12.0 (TID 328). 2515 bytes result sent to driver
23/04/10 16:00:57 INFO TaskSetManager: Starting task 30.0 in stage 12.0 (TID 334) (localhost, executor driver, partition 30, ANY, 5023 bytes) taskResourceAssignments Map()
23/04/10 16:00:57 INFO Executor: Running task 30.0 in stage 12.0 (TID 334)
23/04/10 16:00:57 INFO TaskSetManager: Finished task 24.0 in stage 12.0 (TID 328) in 60344 ms on localhost (executor driver) (19/36)
23/04/10 16:00:57 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
23/04/10 16:00:57 INFO FileOutputCommitter: FileOutput

23/04/10 16:01:15 INFO FileOutputCommitter: Saved output of task 'attempt_202304101558051166772161375833056_0012_m_000025_329' to file:/home/spark/ylv/data/navidata_csv_1/navidata.csv/_temporary/0/task_202304101558051166772161375833056_0012_m_000025
23/04/10 16:01:15 INFO SparkHadoopMapRedUtil: attempt_202304101558051166772161375833056_0012_m_000025_329: Committed
23/04/10 16:01:15 INFO Executor: Finished task 25.0 in stage 12.0 (TID 329). 2515 bytes result sent to driver
23/04/10 16:01:15 INFO TaskSetManager: Starting task 33.0 in stage 12.0 (TID 337) (localhost, executor driver, partition 33, ANY, 5023 bytes) taskResourceAssignments Map()
23/04/10 16:01:15 INFO Executor: Running task 33.0 in stage 12.0 (TID 337)
23/04/10 16:01:15 INFO TaskSetManager: Finished task 25.0 in stage 12.0 (TID 329) in 61494 ms on localhost (executor driver) (22/36)
23/04/10 16:01:15 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
23/04/10 16:01:15 INFO FileOutputCommitter: FileOutput

23/04/10 16:01:19 INFO FileOutputCommitter: Saved output of task 'attempt_20230410155805360052142874823818_0012_m_000023_327' to file:/home/spark/ylv/data/navidata_csv_1/navidata.csv/_temporary/0/task_20230410155805360052142874823818_0012_m_000023
23/04/10 16:01:19 INFO SparkHadoopMapRedUtil: attempt_20230410155805360052142874823818_0012_m_000023_327: Committed
23/04/10 16:01:19 INFO Executor: Finished task 23.0 in stage 12.0 (TID 327). 2515 bytes result sent to driver
23/04/10 16:01:19 INFO TaskSetManager: Finished task 23.0 in stage 12.0 (TID 327) in 97442 ms on localhost (executor driver) (25/36)
23/04/10 16:01:26 INFO FileOutputCommitter: Saved output of task 'attempt_202304101558051927188446984604324_0012_m_000019_323' to file:/home/spark/ylv/data/navidata_csv_1/navidata.csv/_temporary/0/task_202304101558051927188446984604324_0012_m_000019
23/04/10 16:01:26 INFO SparkHadoopMapRedUtil: attempt_202304101558051927188446984604324_0012_m_000019_323: Committed
23/04/10 16:01:26 INFO Exe

In [16]:
spark.sparkContext.getConf().getAll()

[('spark.mongodb.write.connection.uri',
  'mongodb://10.122.6.17:27017/vnw_job.events=None'),
 ('spark.jars',
  '/home/spark/Downloads/rapids-4-spark_2.12-23.02.0.jar,/home/spark/spark/jars/mongo-spark-connector/bson-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/bson-record-codec-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/mongodb-driver-core-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/mongodb-driver-sync-4.8.2.jar,/home/spark/spark/jars/mongo-spark-connector/mongo-spark-connector_2.12-10.1.1.jar,/home/spark/Downloads/spark-tensorflow-connector.jar'),
 ('spark.app.name', 'myApp'),
 ('spark.repl.local.jars',
  'file:///home/spark/Downloads/rapids-4-spark_2.12-23.02.0.jar,file:///home/spark/spark/jars/mongo-spark-connector/bson-4.8.2.jar,file:///home/spark/spark/jars/mongo-spark-connector/bson-record-codec-4.8.2.jar,file:///home/spark/spark/jars/mongo-spark-connector/mongodb-driver-core-4.8.2.jar,file:///home/spark/spark/jars/mongo-spark-connector/mongodb-d

In [3]:
from datetime import datetime

datetime.strftime(datetime.now(), '%Y-01-%mT00:00:00Z')

'2023-01-07T00:00:00Z'